# Import

In [3]:
# 필요한 라이브러리 임포트

import pandas as pd
import random
import os
import numpy as np

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

In [4]:
# 시드값 고정을 위한 클래스 정의.

class CFG:
    SEED = 42

In [5]:
# 시드값 호출을 위한 함수 정의.

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

# Data Load

In [6]:
# 학습 데이터와 테스트 데이터 불러오기.

train = pd.read_csv('open/train.csv')
test = pd.read_csv('open/test.csv')

In [7]:
# trait과 class 피쳐의 특징 파악.

print(train['trait'].value_counts())
print('--------------')

print(train['class'].value_counts())
print('--------------')

2    193
1     69
Name: trait, dtype: int64
--------------
B    114
C     79
A     69
Name: class, dtype: int64
--------------


In [7]:
# 학습 데이터와 테스트 데이터를 동일한 형태로 만들어 주기 위해 전처리 가능한 함수 정의.

def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x

In [8]:
# 함수를 이용해서 학습 데이터와 테스트 데이터 전처리 진행.

train_x, train_y = get_x_y(train)
test_x = get_x_y(test)

# Data Pre-processing
### Label-Encoding

In [9]:
# 머신러닝 학습을 위한 문자열 데이터를 숫자 데이터로 변경하기 위해 LabelEncode() 선언.

class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()

# 유전체의 염기서열을 담고 있는 칼럼들을 snp_col의 이름으로 지정.

snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]
print(snp_col)

['SNP_01', 'SNP_02', 'SNP_03', 'SNP_04', 'SNP_05', 'SNP_06', 'SNP_07', 'SNP_08', 'SNP_09', 'SNP_10', 'SNP_11', 'SNP_12', 'SNP_13', 'SNP_14', 'SNP_15']


In [10]:
# snp_col 칼럼의 데이터들을 snp_data 이름의 배열에 저장.

snp_data = []
for col in snp_col:
    snp_data += list(train_x[col].values)

In [11]:
# 문자열 데이터들을 모두 숫자 데이터로 변경.

train_y = class_le.fit_transform(train_y)
snp_le.fit(snp_data)

LabelEncoder()

In [12]:
for col in train_x.columns:
    if col in snp_col:
        train_x[col] = snp_le.transform(train_x[col])
        test_x[col] = snp_le.transform(test_x[col])

# Model Fit_1

In [13]:
# 학습 데이터로 모델 학습.

clf = RandomForestClassifier(random_state=CFG.SEED)
clf.fit(train_x, train_y)

RandomForestClassifier(random_state=42)

# Inference

In [14]:
preds = clf.predict(test_x)
print('Done.')

Done.


-. 이렇게 학습한 데이터로 테스트 데이터를 모델에 돌려서 정답 추출 후 사이트에 제출하면 0.9622367466의 정확도가 나온다.

# Submission

In [26]:
submit = pd.read_csv('open/sample_submission.csv')

In [27]:
submit['class'] = class_le.inverse_transform(preds)

In [28]:
submit.to_csv('./answer/submit.csv', index=False)

# Model Fit_2

In [24]:
# 하이퍼파리미터 값을 먼저 임의로 설정하여 모델 생성 후 학습하고 GridSearchCV를 통해 최적의 하이퍼 파라미터 값 구하기.

params = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 3, n_jobs = -1)
grid_cv.fit(train_x, train_y)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:  {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 20, 'n_estimators': 100}
최고 예측 정확도: 0.9274


In [25]:
#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 100, 
                                max_depth = 6,
                                min_samples_leaf = 8,
                                min_samples_split = 20,
                                random_state = 0,
                                n_jobs = -1)
rf_clf1.fit(train_x, train_y)
pred = rf_clf1.predict(test_x)

-. 역시 학습한 데이터로 만든 모델에서의 테스트 데이터 예측 결과 제출에 따른 정확도는 0.9523722293 이렇게 나왔다.

# Submission

In [26]:
submit = pd.read_csv('open/sample_submission.csv')

In [27]:
submit['class'] = class_le.inverse_transform(pred)

In [28]:
submit.to_csv('./answer/submit.csv', index=False)